<a href="https://colab.research.google.com/github/h4x4d/narod_parser/blob/main/narod_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prequesties**:

### **Install selenium && other dependencies:**

In [2]:
%%capture
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

pip install selenium

pip install html2text

### **Start selenium browser:**

In [75]:
from selenium import webdriver


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(options=chrome_options)

## **Get site data**:

In [81]:
import requests
import re
from urllib.parse import urljoin
import os
import io
import shutil
from PIL import Image
import html2text
from bs4 import BeautifulSoup
from urllib.request import urlopen
import datetime
import time

In [106]:
os.chdir('/content')
sites = open('Narod.txt')
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(5)
driver.set_window_size(1024, 768)
time_start = datetime.datetime.now()

for index, site_url in enumerate(sites):
  site_url = site_url.strip()
  # variables
  sources = site_url[:site_url.rfind('/')]

  path = sources[sources.find('://') + 3:].replace('/', '_')
  if os.path.exists(path):
    shutil.rmtree(path)
  os.mkdir(path)
  os.chdir(path)


  resp = requests.get(site_url)

  # make all links absolute
  html = resp.text
  absolutize = lambda m: ' src="' + urljoin(sources, m.group(1)) + '"'
  html = re.sub(r' src="([^"]+)"', absolutize, html)
  absolutize2 = lambda m: ' href="' + urljoin(sources, m.group(1)) + '"'
  html = re.sub(r' href="([^"]+)"', absolutize2, html)

  # remove ads
  html = html[:html.rfind('<!-- copyright (t2) -->')]

  # save html
  open('index.html', 'w').write(html)


  # take screenshot
  try:
    driver.get(site_url)
  except:
    pass
  temp = io.BytesIO(driver.get_screenshot_as_png())

  image = Image.open(temp)
  x, y = image.size
  image = image.crop((0, 0, x - 15, y-15))
  image = image.resize((400, 300), Image.Resampling.LANCZOS)
  image.convert('RGB').save('screenshot.jpg')

  # get markdown text file
  open('text.md', 'w').write(html2text.html2text(html))

  # get plain text file

  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  open('text.txt', 'w').write(text)

  open('site_info.txt', 'w').write(
      f'''
      SITE URL: {site_url}

      OTHER INFO...
      '''
  )

  os.chdir('..')
  print(index, site_url, 'time now:', datetime.datetime.now() - time_start)



  print('time all:', datetime.datetime.now() - time_start, (datetime.datetime.now() - time_start) / 100)

0 http://shkolagre.narod.ru/
time on screen: 0:00:03.258945
1 http://franceschool.narod.ru/
time on screen: 0:00:05.531360
2 http://school66pnz.narod.ru/
time on screen: 0:00:05.100129
3 http://site-bsos2.narod.ru/
time on screen: 0:00:04.755451
4 http://www.site-sch354lite.narod.ru/
time on screen: 0:00:02.395927
5 http://borezschool.narod.ru/
time on screen: 0:00:02.866219
6 https://school-number-eight.narod.ru/
time on screen: 0:00:05.228497
7 http://shatkjno.narod.ru/
time on screen: 0:00:05.341866
8 http://verhotorschool.narod.ru/
time on screen: 0:00:05.274742
9 http://kodschool4.narod.ru/
time on screen: 0:00:05.572955
10 http://ars-school4.narod.ru/site_teacher.htm
time on screen: 0:00:03.462173
11 http://sc306.narod.ru/for_teachers.html
time on screen: 0:00:01.202745
12 http://dschi6.narod.ru/
time on screen: 0:00:01.665928
13 http://sh2lysva.narod.ru/index4.html
time on screen: 0:00:05.351129
14 http://sh65-volg.narod.ru/
time on screen: 0:00:03.751607
15 http://vik15603335.n

## **Using data:**

In [107]:
os.chdir('..')

shutil.make_archive('sites', 'zip', '/content')

'/sites.zip'

## **On exit**:

### **Quit driver:**

In [84]:
driver.quit()